In [23]:
import string
import json
import nltk;

import spacy
nlp = spacy.load('en')

In [24]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
# nltk.download('punkt')

stop_words = set(stopwords.words('english')) 

In [25]:

tweets = json.load(open("gg2015.json"))

In [26]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

def find_before(text):
    for i in range(len(text)):
#         print(text[i])
        if re.match('w[io]n?',text[i]):
            return text[i-2]+" "+text[i-1]
    
def find_after(text):
    for i in range(len(text)):
#         print(text[i])
        if text[i]=='goes':
            result = text[i+1]
            if i+2<len(text):
                result = result + " " + text[i+2]
            return result

        if text[i]==':':
            result = ""
            if i+1<len(text):
                result = text[i+1]
            if i+2<len(text):
                result = result +" "+ text[i+2]
            return result

def extract_award(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
#     filtered_sentence = [w for w in word_tokens if not w in stop_words]
    gg_stop_words = ['Globe', 'RT', 'http', 'Golden', 'Globes', 'GoldenGlobes', 'Goldenglobes', 'Goldenglobe', 'gg','golden globes', 'golden globe', 'goldenglobe','goldenglobes','gg2015','gg15','goldenglobe2015','goldenglobe15','goldenglobes2015','goldenglobes15', 'gg2013','gg13','goldenglobe2013','goldenglobe13','goldenglobes2013','goldenglobes13', 'rt', '2013', '2015', '...', '`', 'MTVNews']
    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words and w not in gg_stop_words:
#             w = w.replace('-',",")
            w = w.replace('/',",")
            filtered_sentence.append(w)
    result = ""
    for i in range(len(filtered_sentence)):
#         print(filtered_sentence[i])
        if re.match('goes',filtered_sentence[i]):
#             print('match goes to')
            return find_after(filtered_sentence)

        if re.match('win',filtered_sentence[i]):
            return find_before(filtered_sentence)
#             print('match win')
        if re.match(':', filtered_sentence[i]):
            return find_after(filtered_sentence)
#             print('match :')
    
    return ""
# text = 'Best Actress (Motion Picture - Drama) goes to Jessica Chastain for #ZeroDarkThirty! #GoldenGlobes'
# text = 'best supporting actor goes to christoph waltz in a motion picture for django unchained #goldenglobes'
# print(r)
# result = extract_award(text)
# print('result:',result)

In [27]:
import string
import json
import nltk;
import spacy
nlp = spacy.load('en')

def get_names(text):
    article = nlp(text)
    labels = [x.label_ for x in article.ents]
    [(x.orth_,x.pos_, x.lemma_) for x in [y 
                                      for y
                                      in nlp(text) 
                                      if not y.is_stop and y.pos_ != 'PUNCT']]
    parts_of_speech = dict([(str(x), x.label_) for x in nlp(text).ents])
    names = []
    for (key, value) in parts_of_speech.items() :
        if(value == "PERSON") :
            names.append(key)
    return names

In [28]:
import Levenshtein as lev
def leven(str1, str2):
    Distance = lev.distance(str1.lower(), str2.lower())
    Ratio = lev.ratio(str1.lower(), str2.lower())
#     print(Ratio)
    if Ratio > .95:
        return True
    else:
        return False

In [29]:
import imdb
ia = imdb.IMDb()
def validate_name(name):
    people = ia.search_person(name)
    for p in people:
        str1 = p['name']
        str2 = name
        if leven(str1,str2):
            return True
    return False
# validate_name("kevin costner")


In [30]:
import regex
import json
import re
import regex
from collections import Counter
reg = regex.Regex()
results={}
for movie in reg.award_names:
#     print(movie)
    if movie in reg.people_award:
#         if movie != 'best performance by an actor in a supporting role in a motion picture':
#             continue
        search_term = reg.getRegex(movie)
        
        word_size = 2
#         print(search_term)
        result = []
        for tweet in tweets:  
            text = tweet['text']
            if 'RT' not in tweet['text']:
#                 text = text.lower()
                if re.search(search_term, text):
#                     print(text)
                    winner = extract_award(text)
                    if winner:
                        result.append(winner)
#                         print(winner)
        
#         print(Counter(result).most_common())
        name = ""
        for res in Counter(result).most_common():
            name = res[0]
            if get_names(name) or validate_name(name):
                break
        results[movie] = name
            
for key in results.keys():
    print(key,results[key])
# Counter(result).most_common()    
   

cecil b. demille award George Clooney
best performance by an actress in a motion picture - drama Julianne Moore
best performance by an actor in a motion picture - drama Eddie Redmayne
best performance by an actress in a motion picture - comedy or musical Amy Adams
best performance by an actor in a motion picture - comedy or musical Michael Keaton
best performance by an actress in a supporting role in a motion picture Patricia Arquette
best performance by an actor in a supporting role in a motion picture J.K. Simmons
best director - motion picture Richard Linklater
best performance by an actress in a television series - drama Ruth Wilson
best performance by an actor in a television series - drama Kevin Spacey
best performance by an actress in a television series - comedy or musical Gina Rodriguez
best performance by an actor in a television series - comedy or musical Jeffrey Tambor
best performance by an actress in a mini-series or motion picture made for television Maggie Gyllenhaal
be

In [32]:
# wrong answer (christoph waltz)

